In [2]:
from __future__ import division, unicode_literals

import re
import os
import pandas as pd
import numpy as np

import codecs
from bs4 import BeautifulSoup
import random
import time

In [3]:
dog_data_SF_19 = pd.read_csv('/Users/allisonkelly/Documents/Insight/Petfinder_Project/Model_Data/dog_data_SF_19.csv')

In [2]:
# dog_data_SF_19 = pd.read_csv('/Users/allisonkelly/Documents/Insight/Petfinder_Project/Model_Data/dog_data_STL_19.csv')

In [4]:
dog_data_SF_19 = dog_data_SF_19.drop(['Unnamed: 0'], axis =1)

In [5]:
dog_data_SF_19.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4633 entries, 0 to 4632
Data columns (total 64 columns):
animal_id                     4633 non-null int64
age                           4633 non-null object
attributes.house_trained      4633 non-null int64
attributes.shots_current      4633 non-null int64
attributes.spayed_neutered    4633 non-null int64
attributes.special_needs      4633 non-null int64
breeds.mixed                  4633 non-null int64
breeds.primary                4633 non-null object
coat                          3057 non-null object
colors.primary                2704 non-null object
contact.address.city          4633 non-null object
environment.cats              1812 non-null object
environment.children          2789 non-null object
environment.dogs              3591 non-null object
gender                        4633 non-null object
name                          4633 non-null object
organization_id.1             4633 non-null object
size                          46

In [6]:
dog_data_SF_19.columns

Index(['animal_id', 'age', 'attributes.house_trained',
       'attributes.shots_current', 'attributes.spayed_neutered',
       'attributes.special_needs', 'breeds.mixed', 'breeds.primary', 'coat',
       'colors.primary', 'contact.address.city', 'environment.cats',
       'environment.children', 'environment.dogs', 'gender', 'name',
       'organization_id.1', 'size', 'status', 'url', 'num_photo', 'pub_year',
       'pub_month', 'pub_day', 'missing_description', 'pub_time', 'stat_time',
       'time_in_shelter', 'time_adpt_med', 'tags_cln', 'tags_top', 'num_tags',
       'colors_cat', 'age_Adult', 'age_Baby', 'age_Senior', 'age_Young',
       'gender_Female', 'gender_Male', 'size_Extra Large', 'size_Large',
       'size_Medium', 'size_Small', 'coat_Curly', 'coat_Hairless', 'coat_Long',
       'coat_Medium', 'coat_Short', 'coat_Wire', 'colors_cat_ct_dark',
       'colors_cat_ct_light', 'colors_cat_ct_med', 'pub_month_4',
       'pub_month_5', 'pub_month_6', 'pub_month_7', 'pub_month_8',

### Processing HTML files

In [8]:
path =  "/Users/allisonkelly/Documents/Insight/Petfinder_Project/Pet_Profiles_SF"
os.chdir(path)

profile_raw = [filename for filename in os.listdir(path)]
# profile_raw.remove('.DS_Store')

print(profile_raw[:10])
print(len(profile_raw))

['karolina-45489095', 'skylar-45171826', 'nikko-45938659', 'willow-45239342', 'holden-46073834', 'olivia-44679213', 'monet-45903641', 'teddy-46110106', 'sierra-45414526', 'lettie-44657482']
5345


In [7]:
# html_ex = "/Users/allisonkelly/Documents/Insight/Petfinder_Project/Pet_Profiles/abby-45688357"
# f=open(html_ex, 'r')

# print(re.findall('\-(.*)', html_ex))
# print(re.findall('(?s)<div class=\"u-vr4x\">(.*?)</div>', doc, flags=re.S))


In [9]:
def extract_full_description(html_file):
    html_file_open = open(html_file, "r")
    prof_text = html_file_open.read()
    if len(prof_text) == 0:
        pet_id = re.findall("(?s:.*)\-(.*)", html_file)
        return {"pet_id": pet_id[-1], "full_description": "NULL"}
    else:
        full_description = re.findall(
            '(?s)<div class="u-vr4x">(.*?)</div>', prof_text, flags=re.S
        )
        pet_id = re.findall("(?s:.*)\-(.*)", html_file)
        if len(full_description) != 0:
            return {"pet_id": pet_id[-1], "full_description": full_description[-1]}

In [10]:
dog_data_full_desc = pd.DataFrame(columns=['pet_id', 'full_description'])

In [11]:
for pet in profile_raw:
    full_desc = extract_full_description(pet)
    if full_desc!= 'NULL':
        dog_data_full_desc = dog_data_full_desc.append(full_desc, ignore_index=True)

In [12]:
print(dog_data_full_desc.shape)
print(len(profile_raw))

(5280, 2)
5345


In [13]:
dog_data_full_desc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5280 entries, 0 to 5279
Data columns (total 2 columns):
pet_id              5280 non-null object
full_description    5280 non-null object
dtypes: object(2)
memory usage: 82.6+ KB


In [14]:
len(dog_data_full_desc[dog_data_full_desc['full_description']=='NULL'])

4

In [15]:
dog_data_full_desc['pet_id'].head()

0    45489095
1    45171826
2    45938659
3    45239342
4    46073834
Name: pet_id, dtype: object

In [16]:
dog_data_full_desc['pet_id'].apply(len).describe()

count    5280.0
mean        8.0
std         0.0
min         8.0
25%         8.0
50%         8.0
75%         8.0
max         8.0
Name: pet_id, dtype: float64

In [17]:
dog_data_full_desc['pet_id'] = dog_data_full_desc['pet_id'].astype('int64')

In [18]:
len(dog_data_full_desc['pet_id'].unique())

5280

In [19]:
dog_data_full_desc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5280 entries, 0 to 5279
Data columns (total 2 columns):
pet_id              5280 non-null int64
full_description    5280 non-null object
dtypes: int64(1), object(1)
memory usage: 82.6+ KB


In [20]:
dog_data_full_desc.describe()

,pet_id
count,5.280000e+03
mean,4.534786e+07
std,4.680870e+05
min,4.451789e+07
25%,4.495141e+07
50%,4.531107e+07
75%,4.575338e+07
max,4.612224e+07


In [21]:
dog_data_full_desc.to_csv('dog_data_SF_19_full_desc.csv')

In [22]:
dog_data_SF_19.shape

(4633, 64)

In [23]:
len(dog_data_SF_19['animal_id'].unique())

4621

In [24]:
dog_data_SF_19['animal_id'].head()



0    46110106
1    46106871
2    46100624
3    46096113
4    46095465
Name: animal_id, dtype: int64

In [25]:
dog_data_SF_19['animal_id'].describe()

count    4.633000e+03
mean     4.511837e+07
std      4.562706e+05
min      4.437287e+07
25%      4.471620e+07
50%      4.508719e+07
75%      4.549831e+07
max      4.611011e+07
Name: animal_id, dtype: float64

In [26]:
dog_data_SF_19_full = dog_data_SF_19.merge(dog_data_full_desc, how='inner', left_on='animal_id', right_on='pet_id')

In [27]:
len(dog_data_SF_19.merge(dog_data_full_desc, how='inner', left_on='animal_id', right_on='pet_id'))

3995

In [28]:
dog_data_SF_19_full.shape

(3995, 66)

In [29]:
dog_data_SF_19_full.columns

Index(['animal_id', 'age', 'attributes.house_trained',
       'attributes.shots_current', 'attributes.spayed_neutered',
       'attributes.special_needs', 'breeds.mixed', 'breeds.primary', 'coat',
       'colors.primary', 'contact.address.city', 'environment.cats',
       'environment.children', 'environment.dogs', 'gender', 'name',
       'organization_id.1', 'size', 'status', 'url', 'num_photo', 'pub_year',
       'pub_month', 'pub_day', 'missing_description', 'pub_time', 'stat_time',
       'time_in_shelter', 'time_adpt_med', 'tags_cln', 'tags_top', 'num_tags',
       'colors_cat', 'age_Adult', 'age_Baby', 'age_Senior', 'age_Young',
       'gender_Female', 'gender_Male', 'size_Extra Large', 'size_Large',
       'size_Medium', 'size_Small', 'coat_Curly', 'coat_Hairless', 'coat_Long',
       'coat_Medium', 'coat_Short', 'coat_Wire', 'colors_cat_ct_dark',
       'colors_cat_ct_light', 'colors_cat_ct_med', 'pub_month_4',
       'pub_month_5', 'pub_month_6', 'pub_month_7', 'pub_month_8',

In [30]:
sum(pd.isnull(dog_data_SF_19_full['full_description'])==False)

3995

In [31]:
dog_data_SF_19_full.to_csv(
    "/Users/allisonkelly/Documents/Insight/Petfinder_Project/Model_Data/dog_data_SF_19_full.csv",
    index=False
)